In [ ]:
#This script is an optimization to maximize the energy that the battery charges and discharges for supplying energy to a load.
import matplotlib.pyplot as plt
from pulp import *

# Load profile as an example
load = [32.0, 31.0, 32.0, 31.0, 31.0, 31.0, 38.5, 57.0, 76.0, 121.5, 171.0, 155.0, 117.5, 92.0, 90.0, 108.0, 96.5, 88.0, 78.0, 67.0, 48.0, 39.0, 34.0, 33.0]

# Define max and min of energy and power demanded by load
# Define original battery size
Max_energy=500 #It attempts a C-Rate of 0.5 But it will be changed according to the knee type chart analysis. The value to be chosen is the point of the curve i which there is no more marginal gains in the % in which the battery supplies the load
Max_power=250 

print("Load Power Median Value in each Hours of a day:", (load))

print("Max Power of load: {:.2f} kW".format(Max_power))
print("Min Power of load: {:.2f} kW".format(Min_power))



In [ ]:
!pip install pulp
from pulp import *

# Problem Statement
problem = LpProblem("Battery Optimization", LpMaximize)

# Define variables
hours = range(24)  # 24 hours in a day
power_in = LpVariable.dicts("Power_in", hours, lowBound=0, upBound=Max_power)  # Power input to the battery in each hour
power_out = LpVariable.dicts("Power_out", hours, lowBound=0, upBound=Max_power)  # Power output from the battery in each hour
energy_in = LpVariable.dicts("Energy_in", hours, lowBound=0, upBound=Max_energy)  # Energy charged by the battery in each hour
energy_out = LpVariable.dicts("Energy_out", hours, lowBound=0, upBound=Max_energy)  # Energy discharged by the battery in each hour
soc = LpVariable.dicts("SOC", hours, lowBound=0, upBound=Max_energy)  # State of charge of the battery at the end of each hour
charge_discharge = LpVariable.dicts("Charge_Discharge", hours, cat='Binary')  # Binary variable representing charging or discharging


# Objective function
problem += lpSum(energy_in[hour] for hour in hours)

# Constraints at the beginning of each hour)
for hour in hours:
    if hour == 23:
        problem += power_in[hour] == 0
    if hour == 0:
        problem += soc[hour] == 0
    else:
        problem += soc[hour] == soc[hour-1] + power_in[hour-1] - power_out[hour-1]

    problem += power_in[hour] == energy_in[hour]
    problem += power_out[hour] == energy_out[hour]

    problem += power_in[hour] <= Max_power
    problem += power_out[hour] <= Max_power

    problem += energy_in[hour] <= Max_energy
    problem += energy_out[hour] <= Max_energy

    problem += energy_in[hour] <= charge_discharge[hour] * Max_power
    problem += energy_out[hour] <= (1 - charge_discharge[hour]) * Max_power
    problem += soc[hour] <= Max_energy
# Add load constraints
for hour in hours:
    problem += power_out[hour] <= load[hour]

# Add constraint for SOC to be zero at the beginning and end of the day
problem += soc[0] == 0
problem += soc[23] == 0

# Solve the problem
problem.solve()

# Print the result
print("Status:", LpStatus[problem.status])
print("Optimal Solution:")
for hour in hours:
    print(f"Hour {hour}: Power Input = {power_in[hour].varValue:.2f} kW, Power Output = {power_out[hour].varValue:.2f} kW, SOC = {soc[hour].varValue:.2f} kWh")

# Plot load profile
plt.figure(figsize=(8, 4))
plt.plot(hours, load, marker='o', linestyle='-', color='b')
plt.margins(x=0)
plt.xlabel('Hour')
plt.ylabel('Load (kW)')
plt.title('Load Profile')
plt.grid(True)
plt.show()

# Plot power input to the battery
power_in_values = [power_in[hour].varValue for hour in hours]
plt.figure(figsize=(8, 4))
plt.plot(hours, power_in_values, marker='o', linestyle='-', color='g')
plt.margins(x=0)
plt.margins(y=0)
plt.xlabel('Hour')
plt.ylabel('Power Input (kW)')
plt.title('Power Input to the Battery')
plt.grid(True)
plt.show()

# Plot power output from the battery
power_out_values = [power_out[hour].varValue for hour in hours]
plt.figure(figsize=(8, 4))
plt.plot(hours, power_out_values, marker='o', linestyle='-', color='r')
plt.margins(x=0)
plt.margins(y=0)
plt.xlabel('Hour')
plt.ylabel('Power Output (kW)')
plt.title('Power Output from the Battery')
plt.grid(True)
plt.show()

# Plot state of charge (SOC) of the battery
soc_values = [soc[hour].varValue for hour in hours]
plt.figure(figsize=(8, 4))
plt.plot(hours, soc_values, marker='o', linestyle='-', color='m')
plt.margins(x=0)
plt.margins(y=0)
plt.xlabel('Hour')
plt.ylabel('State of Charge (kWh)')
plt.title('State of Charge of the Battery')
plt.grid(True)
plt.show()

power_difference = [-power_in[hour].varValue + power_out[hour].varValue for hour in hours]
plt.figure(figsize=(8, 4))
plt.plot(hours, power_difference, marker='o', linestyle='-', color='b')
plt.margins(x=0)
plt.margins(y=0)
plt.xlabel('Hour')
plt.ylabel('Power (kW)')
plt.title('Power Input and Output from the Battery')
plt.grid(True)
plt.show()


In [ ]:
total_energy_provided = 0
for hour in hours:
     total_energy_provided += power_out[hour].varValue
print("Total Energy Provided to the Load by the battery:", total_energy_provided, 'kWh')
total_energy_demanded = sum(load)  # Calculate the total energy demanded by the load
print("Total Energy Demanded by the Load:", total_energy_demanded, 'kWh')
print("Percentage of energy supplied by the battery:", total_energy_provided/total_energy_demanded*100, '%')
# Signal output for Hybridization model:
signal = []
for a, b in zip(power_in_values, power_out_values):
    signal.append(a*(-1) + b)

print("The signal to the Hybridization model is:", signal)

In [ ]:
# We try several Energy capacities and find out which delivers the most to the load. Define the range for Max_energy
min_energy = 125
max_energy = 1700
step_energy = 100

# Lists to store the values
max_energy_values = []
percentage_values = []

# Loop through different values of Max_energy
for energy in range(min_energy, max_energy + 1, step_energy):
    # Problem Statement
    problem = LpProblem("Battery Optimization", LpMaximize)
    # Set the value for Max_energy
    Max_energy = energy

    Max_Power=Max_power

    # Problem Statement
    problem = LpProblem("Battery Optimization", LpMaximize)

    # Define variables
    hours = range(24)  # 24 hours in a day
    power_in = LpVariable.dicts("Power_in", hours, lowBound=0, upBound=Max_Power)  # Power input to the battery in each hour
    power_out = LpVariable.dicts("Power_out", hours, lowBound=0, upBound=Max_Power)  # Power output from the battery in each hour
    energy_in = LpVariable.dicts("Energy_in", hours, lowBound=0, upBound=Max_energy)  # Energy charged by the battery in each hour
    energy_out = LpVariable.dicts("Energy_out", hours, lowBound=0, upBound=Max_energy)  # Energy discharged by the battery in each hour
    soc = LpVariable.dicts("SOC", hours, lowBound=0, upBound=Max_energy)  # State of charge of the battery at the end of each hour
    charge_discharge = LpVariable.dicts("Charge_Discharge", hours, cat='Binary')  # Binary variable representing charging or discharging

    # Objective function
    problem += lpSum(energy_in[hour] for hour in hours)  # Maximize the sum of energy input

    # Constraints
    for hour in hours:
        if hour == 23:
            problem += power_in[hour] == 0
        if hour == 0:
            problem += soc[hour] == 0
        else:
            problem += soc[hour] == soc[hour-1] + power_in[hour-1] - power_out[hour-1]

        problem += power_in[hour] == energy_in[hour]
        problem += power_out[hour] == energy_out[hour]

        problem += power_in[hour] <= Max_power
        problem += power_out[hour] <= Max_power

        problem += energy_in[hour] <= Max_energy
        problem += energy_out[hour] <= Max_energy

        problem += energy_in[hour] <= charge_discharge[hour] * Max_power
        problem += energy_out[hour] <= (1 - charge_discharge[hour]) * Max_power
        problem += soc[hour] <= Max_energy

    # Add load constraints
    for hour in hours:
        problem += power_out[hour] <= load[hour]  # Power output cannot exceed the load demand

    # Add constraint for SOC to be zero at the beginning of the day
    problem += soc[0] == 0
    problem += soc[23] == 0


        # Solve the problem
    problem.solve()

    total_energy_provided = 0
    for hour in hours:
        total_energy_provided += power_out[hour].varValue

    max_energy_values.append(Max_energy)
    percentage_values.append(total_energy_provided/sum(load)*100)

# Plot the values
plt.plot(max_energy_values, percentage_values, marker='o')
plt.xlabel("Max Energy")
plt.ylabel("Percentage of Energy Supplied by the Battery")
plt.title("Battery Optimization")
plt.grid(True)
plt.show()
